In [74]:
import numpy as np
import pandas as pd
import scipy
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

def brownian(x0, n, delta = None, loc = None, dt = 1, out = None):
    if not out:
        out = np.ones(n)
    # For each element of x0, generate a sample of n numbers from a
    # normal distribution.
    delta = delta or year_vol/(256**0.5)
    loc = loc or (1 + year_ret)**(1/256)
    r = norm.rvs(loc = loc, size=n, scale=delta*math.sqrt(dt))
    # This computes geometrix Brownian motion by forming the cumulative product of samples. 
    return np.cumprod(r, axis=-1, out = out)*x0

#approximating w/ 4 64 day quarters = 1 year

year_vol = 0.16
year_ret = 0.08
risk_free = 0.03
drisk_free = math.log(1 + risk_free)/256#risk free as the daily compounding rate
d_vol = year_vol/math.sqrt(256)
market = brownian(100, 256)

class Stock:
    def __init__(self, prices = None, price0 = None, exp_vol = None, dividend = 1, market_cor = 0 ):
        self.exp_dvol = exp_vol or d_vol#daily expected vol
        self.dividend = dividend#same every quarter
        self.dividend_every = 64#paid on end of day
        #prices at end of day, assumes started just after dividend paid, 4 dividends/year
        if prices is None: 
            self.true_prices = brownian(price0, n = 30*256) \
                                - np.array([[0]*(64 - 1) + [dividend]]*30*4).flatten()  #subtract dividend on day paid 
        else:
            self.true_prices = prices
        
class Euro_option:
    def __init__(self, strike = 100, premium = 10, lifetime = 256, tp = 'call', asset = None):
        self.strike = strike
        self.premium = premium
        self.lifetime = lifetime
        self.tp = tp
        self.asset = asset or Stock(prices = np.full(lifetime + 1, strike))
        
    def calc_greeks(self, iv = None, days_elapsed = 0, ret_value = False):
        "Calculates the greeks using the BS method"
        if not iv:
            iv = self.asset.exp_dvol
        days_left = self.lifetime - days_elapsed
        spot = self.asset.true_prices[days_elapsed]
        if days_left == 0:
            t = self.lifetime
            if self.tp == 'call':
                return max(0, spot - self.strike)
            elif self.tp == 'put':
                return max(0, self.strike - spot)
        vol_left = iv*days_left**0.5
        d1 = (1 / vol_left*days_left**0.5) * (math.log(spot/self.strike) + (drisk_free + (vol_left**2)/2)*days_left)
        d2 = d1 - vol_left * days_left**0.5
        pv_k = self.strike * math.exp(-drisk_free*days_left)
        if ret_value:
            call = norm.cdf(d1)*spot - norm.cdf(d2)*pv_k
            if self.tp == 'call':
                return call 
            elif self.tp == 'put':
                return pv_k - spot + call
        gamma = norm.pdf(d1)/(spot*days_left**0.5)
        vega = spot*norm.pdf(d1)*days_left**0.5
        theta = -spot*norm.pdf(d1)*vol_left/(2*days_left**0.5)
        if self.tp == 'call':
            delta = norm.cdf(d1)
            theta -= drisk_free*pv_k*norm.pdf(d2)
            rho = days_left*pv_k*norm.pdf(d2)
        elif self.tp == 'put':
            delta = norm.cdf(d1) - 1
            theta += drisk_free*pv_k*norm.pdf(-d2)
            rho = -days_left*pv_k*norm.pdf(-d2)
        return delta, gamma, vega, theta, rho
            
    def make_payoff(option, position = 'long', t = 0):
        stock_prices = np.arange(option.strike - 30, option.strike + 30, 1)
        if 'call' in option.tp:
            payoff = [max(0, i - option.strike) - option.premium for i in stock_prices]
        elif 'put' in option.tp:
            payoff = [max(0, option.strike - i) - option.premium for i in stock_prices]
        if position == 'long':
            pass
        else:
            payoff = [i*-1 for i in payoff]
        fig = plt.figure()
        ax1 = fig.add_subplot(1,1,1) 
        ax1.plot(stock_prices, payoff)
        ax1.set_title(f"Time = {t}")
        return fig
        
call1 = Euro_option(strike = 100, premium = 10, lifetime = 256, tp = 'call')
# call1.make_payoff()
# a = call1.make_payoff(position= 'long')
print(call1.calc_greeks(days_elapsed = 0, ret_value = False))
# plt.plot(call1.asset.true_prices)

(1.0, 0.0, 0.0, -0.0, 0.0)


In [65]:
isnone(1, None)

NameError: name 'isnone' is not defined